In [6]:
import requests
from bs4 import BeautifulSoup
import lxml
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

C:\Users\tandi\AppData\Local\Temp\ipykernel_13852\3954843142.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
class Scraper:
    base_url = "https://www.car.gr"
    url = "https://www.car.gr/api/clsfds/search"
    headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 11; SAMSUNG SM-G973U) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/14.2 Chrome/87.0.4280.141 Mobile Safari/537.36",
    "cookie" : "_gcl_au=1.1.845843389.1739388089; _ga=GA1.1.1746043178.1739388089; _fbp=fb.1.1739388089404.86314614225995742; euconsent-v2=CQMtu0AQMtu0AAKA1AELBcFgAAAAAEPgAAyIAAAVkAJMNCogDLIkJCDQMIIEAKgrCAigQBAAAkDRAQAmDApyBgAusJEAIAUAAwQAgABBkACAAASABCIAIACgQAAQCBQABgAQDAQAMDAAGACwEAgABAdAxTAggECwASMyKBTAhAASCAlsqEEgCBBXCEIs8AiAREwUAAAIABSAAICwWBxJICViQQBcQTQAAEACAQQAFCKTswBBAGbLVXgybRlaYFg-YLntMAyAIgjJyTYgAAAA.YAAAAAAAAAAA; addtl_consent=1~; _clsk=; _clck=; sc=\"7evx65pocW/4mpRoByl7/lrh21I=?_expires=STE3NDIyMzUxMjYKLg==&cookie_id=UycwMjdjZGM3MDA4MWU0YzY5ODE4Y2MxODNhZTdhMGVjMCcKcDAKLg==&name=UydzYycKcDAKLg==\"; _ga_71C4JCCCJZ=GS1.1.1739555836.3.1.1739556727.3.0.297441425",
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-GB,en;q=0.9,el-GR;q=0.8,el;q=0.7,en-US;q=0.6",
    "cache-control": "no-cache",
    "pragma": "no-cache",
    "priority": "u=1, i",
    "referer": "https://www.car.gr/classifieds/cars/?category=15001&pg=3&per-page=50&only-results=0",
    "request-domain": "www.car.gr",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    }
    proxy = {"http": "104.207.37.24:3128"}
    cars = []

    def __init__(self):
        self.session = requests.Session()
        self.querystring = {"category":"15001","pg":1,"per-page":"60"}

    def getPages(self):
        response = self.session.get(url=self.url, headers=self.headers, params=self.querystring)
        data = response.json()
        pagination = data["data"]["results"]["pagination"]
        return round(pagination["total"]/pagination["per_page"]+0.5)

    def fetchCars(self,page):
        cars = []
        self.querystring["pg"] = page
        print(page)
        try:
            response = self.session.get(url=self.url, headers=self.headers, params=self.querystring)
            data = response.json()["data"]["results"]
            cars.extend([{
                "id":car["id"],
                "title":car["title"],
                "make":car["title_parts"]["make"],
                "model":car["title_parts"]["model"],
                "variant":car["title_parts"]["variant"],
                "year":car["title_parts"]["year"],
                "price":car["raw_price"],
                "category":car["category"],
                "registration":car["registration"],
                "crashed":car["crashed"],
                **{
                    feature["key"]:feature["value"].split(" ")[0].replace(".","") for feature in car["key_features"] if feature["key"] in ("fuel_type","gearbox_type","special")
                },
                **{
                    feature["key"]:int(feature["value"].split(" ")[0].replace(".","")) for feature in car["key_features"] if feature["key"] in ("engine_size","engine_power","battery_range","mileage","charging_speed")
                },
                "is_parked":car["is_parked"]} for car in data["rows"]])
            return cars
        except:
            return []


    def run(self):
        pages = self.getPages()
        with ThreadPoolExecutor(max_workers=2) as executor:
            futures = [executor.submit(self.fetchCars, page+1) for page in range(pages-1)]
            for future in as_completed(futures):
                self.cars.extend(future.result())


In [10]:
scraper = Scraper()
scraper.run()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
29
28
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
55
54
57
58
59
56
61
60
62
63
64
66
67
68
65
70
71
72
73
69
74
75
76
78
77
80
79
81
82
83
84
85
86
87
88
89
90
91
92
93
94
96
97
98
99
100
101
95
102
103
104
105
106
107
108
109
111
110
112
113
114
115
116
117
118
119
120
121
122
123
124
125
127
126
129
128
131
130
132
133
135
134
136
137
138
139
141
140
143
142
145
144
147
146
149
148
151
150
153
152
155
154
375
374
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
401
400
402
403
405
406
404
407
408
409
411
410
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
430
429
431
432
433
434
435
436
437
438
439
440
442
441
443
444
445
446
448
447
449
450
451
452
454
453
456
455
458
457
460
459
462
461
463
464
465
466
467
468
469
470
472
471
474
475
473
476
477
478
479
480
481
483
482
484
485
486
487
488
489
490
491
492
493
746
747


KeyError: 'results'

In [9]:
df = pd.DataFrame(scraper.cars)
df.drop_duplicates(keep="first",inplace=True)
df["fuel_type"].value_counts()

fuel_type
Βενζίνη       3029
Πετρέλαιο     2022
Υβριδικό       353
Αέριο(lpg)     233
Ηλεκτρικό       81
Φυσικό          45
Άλλο             3
Name: count, dtype: int64

In [ ]:
start_time = time.time()
proxy ={"http":"185.136.159.181:1080","https":"134.234.152.34:3306"}
base_url = "https://www.car.gr"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}
session = requests.Session()
cars = []
page = 1
url = "https://www.car.gr/api/clsfds/search"
while True:
    # url = base_url + f"/classifieds/cars/?offer_type=sale&category=15001&page={page}&per-page=60"
    querystring = {"category":"15001","pg":page,"per-page":"60"}

    response = session.get(url=url, headers=headers, params=querystring, proxies=proxy)
    print(response.status_code)
    data = response.json()["data"]["results"]
    cars.extend([{
        "id":car["id"],
        "title":car["title"],
        "make":car["title_parts"]["make"],
        "model":car["title_parts"]["model"],
        "variant":car["title_parts"]["variant"],
        "year":car["title_parts"]["year"],
        "price":car["raw_price"],
        "category":car["category"],
        "registration":car["registration"],
        "crashed":car["crashed"],
        "fuel_type":[feature["value"] for feature in car["key_features"] if feature["key"]=="fuel_type"][0],
        **{
            feature["key"]:feature["value"].split(" ")[0].replace(".","") for feature in car["key_features"] if feature["key"] in ("engine_size","engine_power","gearbox_type","battery_range","mileage","charging_speed","special")
        },
        "is_parked":car["is_parked"]} for car in data["rows"]])
    pagination = data["pagination"]
    break
    if round(pagination["total"]/pagination["per_page"]+0.5) == pagination["page"]:
        break
    print(page)
    page += 1

# End the timer after the block
end_time = time.time()

# Calculate the elapsed time in milliseconds
elapsed_time = (end_time - start_time) * 1000
print(f"Execution Time: {elapsed_time:.3f} ms")

ConnectTimeout: HTTPSConnectionPool(host='www.car.gr', port=443): Max retries exceeded with url: /api/clsfds/search?category=15001&pg=1&per-page=60 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000161B89D6AE0>, 'Connection to 134.234.152.34 timed out. (connect timeout=None)'))

In [ ]:
start_time = time.time()
base_url = "https://www.car.gr"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "cookie" : "_gcl_au=1.1.845843389.1739388089; _ga=GA1.1.1746043178.1739388089; _fbp=fb.1.1739388089404.86314614225995742; euconsent-v2=CQMtu0AQMtu0AAKA1AELBcFgAAAAAEPgAAyIAAAVkAJMNCogDLIkJCDQMIIEAKgrCAigQBAAAkDRAQAmDApyBgAusJEAIAUAAwQAgABBkACAAASABCIAIACgQAAQCBQABgAQDAQAMDAAGACwEAgABAdAxTAggECwASMyKBTAhAASCAlsqEEgCBBXCEIs8AiAREwUAAAIABSAAICwWBxJICViQQBcQTQAAEACAQQAFCKTswBBAGbLVXgybRlaYFg-YLntMAyAIgjJyTYgAAAA.YAAAAAAAAAAA; addtl_consent=1~; _clsk=; _clck=; sc=\"7evx65pocW/4mpRoByl7/lrh21I=?_expires=STE3NDIyMzUxMjYKLg==&cookie_id=UycwMjdjZGM3MDA4MWU0YzY5ODE4Y2MxODNhZTdhMGVjMCcKcDAKLg==&name=UydzYycKcDAKLg==\"; _ga_71C4JCCCJZ=GS1.1.1739555836.3.1.1739556727.3.0.297441425",
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-GB,en;q=0.9,el-GR;q=0.8,el;q=0.7,en-US;q=0.6",
    "cache-control": "no-cache",
    "pragma": "no-cache",
    "priority": "u=1, i",
    "referer": "https://www.car.gr/classifieds/cars/?category=15001&pg=3&per-page=50&only-results=0",
    "request-domain": "www.car.gr",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
}
session = requests.Session()
features_dict = {
    'Αυτονομία μπαταρίας':"battery_miles",
    'Ιδιαιτερότητες':"special",
    'Ιπποδύναμη':"hp",
    'Καύσιμο':"fuel",
    'Κυβικά':"cc",
    'Σασμάν':"transmission",
    'Χιλιόμετρα':"kilometers",
    'Χρονολογία':"manufacture_year",
    'Χρόνος φόρτισης / γρήγορης φόρτισης':"charging_speed",
    "Ισχύς κινητήρα":"engine_power"
}
data = []
page = 1
while True:
    # url = base_url + f"/classifieds/cars/?offer_type=sale&category=15001&page={page}&per-page=60"
    url = f"https://www.car.gr/api/clsfds/search"
    querystring = {"category":"15001","pg":page,"per-page":"60"}

    response = session.get(url=url, headers=headers, params=querystring)
    json = response.json()["data"]["results"]["rows"]
    data.extend([{
        "id":car["id"],
        "title":car["title"],
        "make":car["title_parts"]["make"],
        "model":car["title_parts"]["model"],
        "variant":car["title_parts"]["variant"],
        "year":car["title_parts"]["year"],
        "price":car["raw_price"],
        "category":car["category"],
        "registration":car["registration"],
        "crashed":car["crashed"],
        "fuel_type":[feature["value"] for feature in car["key_features"] if feature["key"]=="fuel_type"][0],
        **{
            feature["key"]:feature["value"].split(" ")[0].replace(".","") for feature in car["key_features"] if feature["key"] in ("engine_size","engine_power","gearbox_type","battery_range","mileage","charging_speed","special")
        },
        "is_parked":car["is_parked"]} for car in json])
        
    # soup = BeautifulSoup(response.text, 'lxml')

    # cars = [li.a for li in soup.ol.find_all("li")]


    # data.extend([
    #     {
    #         "link": base_url + car["href"],
    #         "name": car.h2.text.strip(),
    #         "price": int(car.find("div", attrs={"class": "current-price"}).text.replace(".", "").replace("€", "")),
    #         "type": car.findAll("span", attrs={"class": "tw-text-grey-600"})[-1].text.strip(),
    #         "place": car.findAll("span")[-6].text.strip(),
    #         **{
    #             features_dict[feature["title"]]: (
    #                 feature.text.strip().split(" ")[0].replace(".","") if feature["title"] in ('Ιπποδύναμη','Κυβικά','Χιλιόμετρα')
    #                 else feature.text.strip())
    #             for feature in car.findAll("span", attrs={"class": "key-feature"})
    #         }
    #     }
    #     for car in cars
    # ])


# End the timer after the block
end_time = time.time()

# Calculate the elapsed time in milliseconds
elapsed_time = (end_time - start_time) * 1000
print(f"Execution Time: {elapsed_time:.3f} ms")

https://www.car.gr/api/clsfds/search
200
Execution Time: 718.825 ms


In [ ]:
data[0].keys()

dict_keys(['geolocation', 'marketplace', 'leasing', 'price_per_unit', 'consent', 'is_new', 'foreign_number_plate', 'states', 'label', 'own', 'model_stub', 'thumbs', 'registration', 'seo', 'paid_features', 'category_ids', 'dealer_cls', 'id', 'category', 'price_debatable', 'user_id', 'thumb', 'certificate', 'title', 'views', 'without_vat', 'descriptive_title', 'seller', 'has_youtube', 'created', 'external_link', 'seo_json_ld', 'clicks', 'url_slug', 'seo_url', 'mileage', 'has_finance', 'key_features', 'is_parked', 'price', 'title_parts', 'short_description', 'crashed', 'address_long', 'thumbs_patterns', 'raw_price', 'address', 'price_parked', 'deleted_date', 'offertype', 'has_shorts', 'has_certificate', 'engine_size', 'is_deleted', 'has_leasing', 'thumb_pattern', 'discount', 'from_promotion', 'finance_options', 'dealers_only', 'modified', 'all_final_category_ids', 'length', 'has_360photos', 'vtype_xyma_or_part', 'promotion', 'fueltype', 'has_photos', 'parked'])

In [ ]:
data[0]["category"]

'Αυτοκίνητο » Κόμπακτ/Hatchback'